# About: Using Selenium

Jupyter NotebookからSeleniumを使ってみる例です。

1. GitHubリポジトリ https://github.com/yacchin1205/jupyter-with-jenkins を開き
2. `conf`ディレクトリの中身を覗く
3. サーバーの応答時間を可視化する

ことをしてみます。

# 準備

必要なライブラリをインポートします。

In [ ]:
import time
import os
from IPython.display import Image

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

WebDriverを準備します。本環境にはChrome Driverのみインストールしています。(Firefoxとか、Pull Requestもらえると嬉しかったり)

> サーバーの応答時間を可視化するために、Chromeを前提としたパラメータを指定しています。

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--lang=ja')

# DriverからPerfomance Logを取得し、これをHARとして整形する
# 参考: https://stackoverflow.com/questions/18075645/export-har-using-chromedriver
perf_log_prefs = dict(traceCategories='browser,devtools.timeline,devtools', enableNetwork=True, enablePage=True)
chrome_options.add_experimental_option('perfLoggingPrefs', perf_log_prefs)

#caps = webdriver.DesiredCapabilities.CHROME.copy() 
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL', 'browser': 'ALL'})

driver = webdriver.Chrome(options=chrome_options)

スクリーンショットを取得するための一時ディレクトリを準備しておきます。

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

# テストの実施

## ブラウザを開く

WebDriverを使って、URLを指定してブラウザを開きます。

In [ ]:
url = 'https://github.com/yacchin1205/jupyter-with-jenkins'
driver.get(url)

# Output the screenshot
driver.save_screenshot(os.path.join(work_dir, 'sample.png'))
Image(os.path.join(work_dir, 'sample.png'))

## 画面遷移の確認

`conf`ディレクトリの中身を覗いてみます。

In [ ]:
elem = driver.find_element(By.XPATH, '//a[text() = "conf"]')
elem.click()

driver.save_screenshot(os.path.join(work_dir, 'sample.png'))
Image(os.path.join(work_dir, 'sample.png'))

クリック後即confディレクトリが表示されるわけではないので... `time.sleep`等で適宜待機してから確認するのが良いかもしれません。

In [ ]:
time.sleep(5)

driver.save_screenshot(os.path.join(work_dir, 'sample.png'))
Image(os.path.join(work_dir, 'sample.png'))

## 要素が存在しているかチェック

このように画面遷移をさせながら、適宜予期した要素が存在しているかなど日々チェックできると安心かなと。

以下のコードは期待した要素が存在していないと`NoSuchElementException`で失敗します。

In [ ]:
# If the expected file does not exist, it fails with NoSuchElementException
driver.find_element(By.XPATH, '//a[text() = "jupyter_notebook_config.py"]')

# 応答速度確認

この例では [harファイル](https://toolbox.googleapps.com/apps/har_analyzer/?lang=ja) を取得してみます。HAR Analyzer等のアプリケーションを利用することで、Webサイトのパフォーマンスを詳細に分析することができます。

In [ ]:
import requests
import json

# DriverからPerfomance Logを取得し、これをHARとして整形する
# 参考: https://stackoverflow.com/questions/18075645/export-har-using-chromedriver
def get_perf_entry_logs(driver):
    r = []
    for log in driver.get_log('performance'):
        r.append(json.loads(log['message'])['message'])
    return r

def load_har(driver):
    r = requests.get('https://cdn.jsdelivr.net/gh/Ankit3794/chrome_har_js@master/chromePerfLogsHAR.js')
    r.encoding = r.apparent_encoding
    perf_logs_har_script = r.text

    driver.execute_script(perf_logs_har_script);
    har_body = driver.execute_script("return module.getHarFromMessages(arguments[0])", json.dumps(get_perf_entry_logs(driver)))
    return json.loads(har_body)

har_data = load_har(driver)
har_data

URLごとにtimeの項目を抽出してみます。

In [ ]:
import pandas as pd

perf_df = pd.DataFrame([(e['request']['url'], e['time']) for e in har_data['log']['entries']], columns=['URL', 'time_ms'])
perf_df

time_msの分布をヒストグラムで出しておくと、応答時間の傾向を把握できるかもしれません。

In [ ]:
perf_df['time_ms'].hist()

harファイルは日付をつけて`har` ディレクトリに保存しておくことにします。
あとで問題が起きた時に分析したり、長期的なトレンドを分析するのに役立つかもしれません。

In [ ]:
from datetime import datetime

har_files_path = './har'
!mkdir -p {har_files_path}

# ファイル名はyyyymmddhhmmssとする - 環境のTimezoneに注意
filename = 'selenium-' + datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
with open(os.path.join(har_files_path, f'{filename}.json'), 'w') as f:
    f.write(json.dumps(har_data))
!ls -la {har_files_path}

# 後始末

忘れず一時ファイルは削除しておきましょう。

In [ ]:
!rm -fr {work_dir}